In [ ]:
###############################################################################
# Parameers
###############################################################################
lp_root = ['/home/atiroms/Documents','D:/atiro','D:/NICT_WS','/Users/smrt']
id_calendar = 'ht4svlr03krt7jcqho5guou32c@group.calendar.google.com'
fyear = 2025
season = 'summer'
constant_rank = 1.0
#constant_rank = 10.0

###############################################################################
# Libraries
###############################################################################
import pandas as pd, numpy as np, datetime as dt
import os, datetime
from pulp import *
from ortoolpy import addbinvars
from googleapiclient.discovery import build
from script.helper import *

###############################################################################
# Load and modify data
###############################################################################
p_root, p_month, p_data = prep_dirs(lp_root = lp_root, year_plan = fyear, month_plan = season, prefix_dir = 'asgn')
path_form = '/dutyshift/result/' + str(fyear) + '/' + season + '/' +  str(fyear) + season
d_availability_src = read_form_response(p_root, path_form)
d_availability = d_availability_src.copy()

# Pick up newest of each member
l_member = sorted(list(set(d_availability['お名前（敬称略）'].tolist())))
l_d_availability = []
for member in l_member:
    d_availability_temp = d_availability[d_availability['お名前（敬称略）'] == member]
    d_availability_temp = d_availability_temp.sort_values(by = ['Timestamp'], ascending = False)
    d_availability_temp = d_availability_temp.iloc[0]
    l_d_availability.append(d_availability_temp)
d_availability = pd.DataFrame(l_d_availability)

d_availability = d_availability.drop(['Timestamp'], axis = 1)
l_week = d_availability.columns[1:]
l_week = [col.split('[')[1].split(' - ')[0] for col in l_week]
d_availability.columns = ['name_jpn_full'] + l_week


###############################################################################
# Load and modify data
###############################################################################

d_member = pd.read_excel(os.path.join(p_month, 'member.xlsx'), sheet_name = 'team')
d_team = d_member.drop(['name_jpn', 'name_jpn_full', 'rank'], axis = 1)
d_team = d_team.replace('-', '')
l_team = [team for team in sorted(list(set(d_team.iloc[:,1:].values.flatten().tolist()))) if team != '']
d_member = d_member[['id_member', 'name_jpn_full', 'email', 'rank']]
d_member['name_jpn_full'] = [name.replace('　',' ') for name in d_member['name_jpn_full']]

d_availability = pd.merge(d_availability, d_member, on = 'name_jpn_full')
d_availability = d_availability[['id_member'] + l_week]
d_availability = d_availability.sort_values(by = ['id_member'])
d_availability.index = d_availability['id_member'].tolist()
l_member = d_availability['id_member'].tolist()

d_absence = d_availability.iloc[:,1:] == '学会・出張'
d_absence['id_member'] = l_member
d_absence = d_absence[['id_member'] + l_week]

d_availability = d_availability.replace('第１希望', 1)
d_availability = d_availability.replace('第２希望', 2)
d_availability = d_availability.replace('第３希望', 3)
d_availability = d_availability.replace('第４希望', 4)
d_availability = d_availability.replace('学会・出張', np.nan)
d_availability = d_availability.replace('第１希望 | 学会・出張', np.nan)


###############################################################################
# Initialize assignment problem and model
###############################################################################
# Initialize model to be optimized
prob_assign = LpProblem()

# Binary assignment variables to be optimized
dv_assign = pd.DataFrame(np.array(addbinvars(len(l_member), len(l_week))),
                         index = l_member, columns = l_week)


###############################################################################
# One assignment per member
###############################################################################
for member in l_member:
    prob_assign += (lpSum(dv_assign.loc[member,:]) == 1)

###############################################################################
# Do not assign to unavailable week
###############################################################################
prob_assign += (lpDot(np.isnan(d_availability.iloc[:,1:]).to_numpy(), dv_assign.to_numpy()) <= 0)


###############################################################################
# Maximum of 3 members per week
###############################################################################
for week in l_week:
    prob_assign += (lpSum(dv_assign[week]) + d_absence[week]) <= 4


###############################################################################
# Avoid members from the same team
###############################################################################
for week in l_week:
    for team in l_team:
        lv_assign = dv_assign[week].tolist()
        l_absence = d_absence[week].tolist()
        l_team_week = d_team.loc[d_team['id_member'].isin(l_member), week].tolist()
        l_team_week = [t == team for t in l_team_week]
        prob_assign += (lpDot(lv_assign, l_team_week) + np.dot(l_absence, l_team_week) <= 1)


###############################################################################
# Define objective function to be minimized
###############################################################################
d_optimality = d_availability.replace(np.nan, 0)
# Convert [1,2,3,4] to [1,2,4,7]
d_optimality.iloc[:,1:] = d_optimality.iloc[:,1:].replace(4,7).replace(3,4)
#l_rank = [[x * constant_rank / d_optimality.shape[0]] * (d_optimality.shape[1] - 1) for x in range(d_optimality.shape[0])]
l_rank = [[x * constant_rank] * (d_optimality.shape[1] - 1) for x in d_member['rank'].tolist()]
d_rank = pd.DataFrame(l_rank, index = d_optimality.index, columns = d_optimality.columns[1:])
d_optimality.iloc[:,1:] = d_optimality.iloc[:,1:] + d_rank
prob_assign += (lpDot(dv_assign.to_numpy(), d_optimality.iloc[:,1:].to_numpy()))


###############################################################################
# Solve problem
###############################################################################
# Print problem
#print('Problem: ', problem)

# Solve problem
prob_assign.solve()
v_objective = value(prob_assign.objective)
print('Solved: ' + str(LpStatus[prob_assign.status]) + ', ' + str(round(v_objective, 2)))

###############################################################################
# Extract result
###############################################################################
d_assign = pd.DataFrame(np.vectorize(value)(dv_assign),
                        columns = dv_assign.columns, index = dv_assign.index).astype(bool)
l_assign = d_assign.apply(lambda row: row[row == True].index.tolist()[0], axis=1)
d_assign_member = pd.DataFrame({'id_member': l_member, 'md_start': l_assign})
d_assign_member = pd.merge(d_assign_member, d_member, on = 'id_member')
d_assign_member['m_start'] = d_assign_member['md_start'].apply(lambda x: int(x.split('/')[0]))
d_assign_member['d_start'] = d_assign_member['md_start'].apply(lambda x: int(x.split('/')[1]))
d_assign_member['y_start'] = fyear
d_assign_member.loc[d_assign_member['m_start'] < 4, 'y_start'] = fyear + 1
d_assign_member['unix_start'] = [dt.datetime(year = year, month = month, day = date).timestamp() for year, month, date in zip(d_assign_member['y_start'].tolist(),d_assign_member['m_start'].tolist(), d_assign_member['d_start'].tolist()) ]
d_assign_member['unix_end'] = d_assign_member['unix_start'] + dt.timedelta(days = 5).total_seconds() - 60
d_assign_member['m_end'] = d_assign_member['unix_end'].apply(lambda x: dt.datetime.fromtimestamp(x).month)
d_assign_member['d_end'] = d_assign_member['unix_end'].apply(lambda x: dt.datetime.fromtimestamp(x).day)
d_assign_member['md_end'] = [str(month) + '/' + str(date) for month, date in zip(d_assign_member['m_end'].tolist(), d_assign_member['d_end'].tolist()) ]
d_assign_member['duration'] = [start + ' - ' + end for start, end in zip(d_assign_member['md_start'].tolist(), d_assign_member['md_end'].tolist())]
d_assign_member['optimality'] = [int(x) for x in (d_availability.iloc[:,1:] * d_assign).sum(axis = 1).tolist()]

d_assign_week = pd.DataFrame({'week': l_week, 'id_member': d_assign.apply(lambda row: row[row].index.to_list(), axis = 0)})
d_assign_week.index = range(len(d_assign_week))
for id, row in d_assign_week.iterrows():
    l_id_member = row['id_member']
    week = row['week']
    s_id_member_absence = d_absence[week]
    l_id_member_absence = s_id_member_absence[s_id_member_absence].index
    if len(l_id_member) == 0:
        str_member = 'なし'
    else:
        l_str_member = d_member.loc[d_member['id_member'].isin(l_id_member), 'name_jpn_full'].tolist()
        str_member = ', '.join(l_str_member)
    d_assign_week.loc[id, 'member'] = str_member
    if len(l_id_member_absence) == 0:
        str_member_absence = 'なし'
    else:
        l_str_member_absence = d_member.loc[d_member['id_member'].isin(l_id_member_absence), 'name_jpn_full'].tolist()
        str_member_absence = ', '.join(l_str_member_absence)
    d_assign_week.loc[id, 'member_absence'] = str_member_absence

for p_save in [p_month, p_data]:
    d_assign.to_csv(os.path.join(p_save, 'assign.csv'), index = False)
    d_assign_member.to_csv(os.path.join(p_save, 'assign_member.csv'), index = False)
    d_assign_week.to_csv(os.path.join(p_save, 'assign_week.csv'), index = False)

print(d_assign_week)


In [ ]:

###############################################################################
# Handle Credentials and token
###############################################################################

creds = prep_api_creds(p_root, l_scope = ['https://www.googleapis.com/auth/calendar']
                       )
service_calendar = build('calendar', 'v3', credentials = creds)


###############################################################################
# Create and share events
###############################################################################
####
# Used for testing
#d_assign_member = d_assign_member.loc[d_assign_member['id_member'] == 5]
####
l_result_event = []


for _, row in d_assign_member.iterrows():
    name_member = row['name_jpn_full'].replace('　',' ')
    email = row['email']
    if season == 'summer':
        summary = '東大病院夏季休暇'
        description = name_member + '先生東大病院夏季休暇' +'\nhttps://github.com/atiroms/dutyshift で自動生成'
    elif season == 'winter':
        summary = '東大病院冬季休暇'
        description = name_member + '先生東大病院冬季休暇' +'\nhttps://github.com/atiroms/dutyshift で自動生成'
    t_start = dt.datetime(year = row['y_start'], month = row['m_start'], day = row['d_start'], hour = 0, minute = 0).isoformat()
    t_end = dt.datetime(year = row['y_start'], month = row['m_end'], day = row['d_end'], hour = 23, minute = 59).isoformat()
    body_event = {'summary': summary,
                  'start': {'dateTime': t_start, 'timeZone': 'Asia/Tokyo'},
                  'end': {'dateTime': t_end, 'timeZone': 'Asia/Tokyo'},
                  'attendees': [{'email': email}],
                  #'attendees': [{'email': email, 'displayName':name_member}],
                  #'attendees': [{'email': email, 'responseStatus':'accepted'}],
                  'description': description
                  }
    result_event = service_calendar.events().insert(calendarId = id_calendar, body = body_event).execute()
    l_result_event.append(result_event)
